HW 1 - Ebay Wrangling  
Karol Kusmierczuk 

Import pandas and read in files to work with:

In [1]:
#import pandas
import pandas as pd
import re

#read in file
original_ebay_df = pd.read_csv('/Users/karol/Downloads/Data Management/HW 1/EbayPcLaptopDataUnclean.csv')
price_df = pd.read_csv('/Users/karol/Downloads/Data Management/HW 1/EbayPcLaptopPriceData.csv')
ebay_df = original_ebay_df


Exploring the data by checking the data types, size of data, and other details:

In [2]:
print(ebay_df.shape)
print(ebay_df.dtypes)
print(ebay_df.describe())
print(ebay_df.head())

(6676, 23)
Item Number                        int64
Brand                             object
Rating                            object
Ratings Count                    float64
Processor                         object
Screen Size                       object
Manufacturer Color                object
Color                             object
Ram Size                          object
SSD Capacity                      object
GPU                               object
Processor Speed                   object
Type                              object
Release Year                      object
Maximum Resolution                object
Model                             object
OS                                object
Hard Drive Capacity               object
Country Region Of Manufacture     object
Storage Type                      object
Features                          object
Condition                         object
Seller Note                       object
dtype: object
        Item Number  Ratings Cou

In [3]:
print(ebay_df.dtypes)

Item Number                        int64
Brand                             object
Rating                            object
Ratings Count                    float64
Processor                         object
Screen Size                       object
Manufacturer Color                object
Color                             object
Ram Size                          object
SSD Capacity                      object
GPU                               object
Processor Speed                   object
Type                              object
Release Year                      object
Maximum Resolution                object
Model                             object
OS                                object
Hard Drive Capacity               object
Country Region Of Manufacture     object
Storage Type                      object
Features                          object
Condition                         object
Seller Note                       object
dtype: object


Removing duplicates from the price file, adding minimum and maximum price ranges to the main file:  
I added the price ranges since some product did not have just one price assigned to them and this way more of the orignial data stays rather than finding averages

In [4]:
#Add in price and make new columns for price ranges
price_df_unique = price_df.drop_duplicates()

price_df_unique['Min Price'] = price_df_unique['Price'].apply(lambda x: float(x.split()[0].replace('$', '').replace(',', '')))
price_df_unique['Max Price'] = price_df_unique['Price'].apply(lambda x: float(x.split(' to ')[-1].replace('$', '').replace(',', '')))

merge_price = price_df_unique[['Item Number', 'Min Price', 'Max Price']]

ebay_df = pd.merge(ebay_df, merge_price, on='Item Number', how='left')
#ebay_df

<ipython-input-4-e9c69e9d13f9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df_unique['Min Price'] = price_df_unique['Price'].apply(lambda x: float(x.split()[0].replace('$', '').replace(',', '')))
<ipython-input-4-e9c69e9d13f9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df_unique['Max Price'] = price_df_unique['Price'].apply(lambda x: float(x.split(' to ')[-1].replace('$', '').replace(',', '')))


Checking the percentage of data missing in each column:

In [5]:
ebay_df.isna().sum() / len(ebay_df) * 100

Item Number                       0.000000
Brand                            38.615938
Rating                           95.641102
Ratings Count                    95.641102
Processor                        41.446974
Screen Size                      40.638107
Manufacturer Color               97.153984
Color                            67.121031
Ram Size                         67.450569
SSD Capacity                     57.399641
GPU                              53.235470
Processor Speed                  59.152187
Type                             49.835231
Release Year                     87.807070
Maximum Resolution               62.088077
Model                            51.842421
OS                               53.520072
Hard Drive Capacity              64.439784
Country Region Of Manufacture    96.929299
Storage Type                     63.406231
Features                         61.698622
Condition                         9.376872
Seller Note                      77.741162
Min Price  

Tried to fill in any missing values in 'Color' with data from 'Manufacturer Color' since 'Color' has more data and I do not see a need for both of these columns

In [6]:
ebay_df['Color'] = ebay_df['Color'].fillna(ebay_df['Manufacturer Color'])

Running the code above made no difference, so it will support my decision to drop 'Manufacturer Color'

Deleted the following columns based on the following reseasons:  
Seller Note - Too much messy data  
Features - Too much messy data  
Manufactoring Color - Missing almost entire column of data and there is another color column with more data  
Rating - Missing almost entire column of data  
Ratings Count - Missing almost entire column of data  
Country Region of Manufacture - Missing almost entire column of data, also does not seem to be an important column for laptop buying decision making

In [7]:
#ebay_df.drop(columns = ['Seller Note', 'Features', 'Manufacturer Color', 'Rating', 'Ratings Count', 'Country Region Of Manufacture' ], inplace = True)
ebay_df.drop(columns = ['Seller Note', 'Features', 'Manufacturer Color', 'Country Region Of Manufacture'], inplace = True)

Remove missing data based on what is missing from the 'Model' column:  
I belive this is the most important column as based on this column, you could figure out the rest of the information with some research (along with knowing the brand which I would say is the second most important column, but if the model is mentioned then the brand should be there anyway)

Remove missing data:
Altough

In [8]:
ebay_df.dropna(subset=['Model'], inplace = True)
#ebay_df.dropna(inplace = True)
len(ebay_df)

3215

Added the column "Refurbished" which tells if the product was refurbished or not:

In [9]:
ebay_df['Release Year'] = ebay_df['Release Year'].astype(str)
ebay_df['Refurbished'] = ebay_df['Release Year'].apply(lambda x: 'Yes' if 'refurbished' in x.lower() else 'No')

Cleaned the 'Year Released' column by first looking at the uniques and then replacing data where needed:

In [10]:
#shows the unique entries for this column
ebay_df['Release Year'].unique()

array(['2021', 'Refurbished in 2023', '2019', 'nan', '2015', '2017',
       '2020', '2018', '2023', '2013', '2022', '2014', '2016', '2007',
       '2009', 'Acer Aspire V3-571G', '2010', '2012',
       '2022 (Refurbished)', '0', '2006', 'Various', '2008',
       'see manufacturer', 'na', '2023 Refurbished',
       '2020 (Refurbished By MAR)', 'Cyrstal Clear Touchscreen',
       '2024 REFURBISH DATE', '2022 Refurbished', 'Updated to 2023',
       'N/a', '2005', '6/11/2021', 'Any', 'Unknown',
       'Seller Refurbished: 2022', 'Refurbished 2023', '2000'],
      dtype=object)

In [11]:
#Since refurbished is not the original release year, it is unknown
ebay_df['Release Year'].replace(to_replace=['Refurbished in 2023', 'Acer Aspire V3-571G', '2022 (Refurbished)', '0', 'Various', 'see manufacturer', 'na',
       '2023 Refurbished', '2020 (Refurbished By MAR)',
       'Cyrstal Clear Touchscreen', '2024 REFURBISH DATE',
       '2022 Refurbished', 'Updated to 2023', 'N/a', 'Any', 'Seller Refurbished: 2022', 'Refurbished 2023'], 
       value='Unknown', inplace=True)
ebay_df['Release Year'].replace(to_replace=['6/11/2021'], 
       value='2021', inplace=True)
#fill in remaining missing data with unknown
ebay_df['Release Year'] = ebay_df['Release Year'].astype(str).replace('nan', 'Unknown')

Cleaned the 'Hard Drive Capacity' column by first looking at the uniques and then replacing data where needed:

In [12]:
ebay_df['Hard Drive Capacity'].unique()

array(['512 GB SSD', '2 TB', '500 GB', '16 GB', 'NO HDD', nan, '16GB',
       '512 GB', '256 GB', '128 GB', '1 TB', 'Up to 2TB', '512GB SSD',
       '12 GB', '8 GB', '320 GB', 'No HDD', 'Not Included', '180 GB',
       '32 GB', 'NO', 'No', '128GB', '160 GB', '64 GB', '500GB', '240 GB',
       '200 GB', 'SSD', '80GB-100 GB', '300 GB', '1tb', '1TB', '0',
       '160GB', '4 GB', '750 GB', '400GB', '250 GB', '1 GB',
       '120GB 240GB 512GB 1TB SELECT FROM ABOVE', '2 GB', '64gb',
       '720 GB', '128GB,  256GB,  500GB,  1TB,  2TB', '64GB', '1TB SSD',
       'Not Applicable', '512gb', 'Up to 1 TB', 'SSD ONLY', '32GB',
       '120 GB', '0 GB', 'SEE SSD', '80GB', '4 TB', 'up to 1 TB',
       '256GB/512GB/1TB SSD (as selected)', '256GB - 500GB - 1TB - 2TB',
       '80 GB', '64GB eMMC', '256 GB SSD', '256 GBSSD', '128 SSD',
       '2TB m.2 NvMe SSD', '16 GB SSD', '512GB', '640 GB', 'non', 'N/a',
       '3 GB', 'not included', '20 GB', '800 GB', '256 GB 512 GB 1TB 2TB',
       '465 GB', '256GB

In [13]:
#made the assumption that if an entry had "up to x GB" that it can hold that amount 
#since some of the data had a range of options, it is unknown what the specific storage amount is
ebay_df['Hard Drive Capacity'].replace(to_replace=['512 GB SSD', '512', '512gb', '512GB SSD', '512GB', '512 GB SSD Solid State Drive', '512 GB NVMe M.2 SSD Solid State Drive'], 
       value='512 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['No HDD', 'Not Included', 'NO', 'No', 'Not Applicable', '120GB 240GB 512GB 1TB SELECT FROM ABOVE', '0 GB', 'SEE SSD'
       '3500.16 GB', '256GB/512GB/1TB SSD (as selected)', '256GB - 500GB - 1TB - 2TB', 'non', 'N/a', 'not included', '256 GB 512 GB 1TB 2TB', 'SSD only', '1',
       '256 GB 512 GB 1 TB 2 TB', '0','80GB-100 GB','SSD','3500.16 GB','SSD Only','128GB,  256GB,  500GB,  1TB,  2TB', '64GB', '1TB SSD','250 GB or More','250 GB- 1 TB', '128 GB 256 GB 512 GB 1TB 2TB','SEE SSD','SSD ONLY', '0GB', '250 GB - 1 TB', 'Non', '.', 'Not included', 'N.A.', "Don't Apply", '120 GB or More',
       '320 GB- 500 GB', 'NO HDD'], 
       value='None/Unknown', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['500GB Solid State Hybrid Drive', '500 GB Large storage memory', '500GB', '500 GB 1 TB 2 TB'], 
       value='500 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['Up to 2TB', 'Up to 2 TB', '2TB m.2 NvMe SSD', '2 TB SSD Solid State Drive', '2TB'], 
       value='2 TB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['16GB', '16 GB SSD'], 
       value='16 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['160GB'], 
       value='160 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['32GB'], 
       value='32 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['320GB'], 
       value='320 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['80GB'], 
       value='80 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['400GB'], 
       value='400 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['1 TB SSD Solid State Drive', '1tb nvme', 'Up to 1 TB', '1tb', '1TB', 'up to 1 TB'], 
       value='1 TB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['256 GB SSD', '256 GBSSD', '256GB', '81GB', '256gb'], 
       value='256 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['128GB', '128 GB (gigabyte)', '128 SSD'], 
       value='128 GB', inplace=True)
ebay_df['Hard Drive Capacity'].replace(to_replace=['64gb', '64GB eMMC', '64GB SSD'], 
       value='64 GB', inplace=True)
#fill in remaining missing data with unknown
ebay_df['Hard Drive Capacity'] = ebay_df['Hard Drive Capacity'].astype(str).replace('nan', 'Unknown')


Created a new column if the laptop has a touchscreen:

In [14]:
#function that checks if the word touchscreen was mentioned in the 'Screen Size' column
def check_touch_screen(value):
    if isinstance(value, str):
        value_lower = value.lower()
        if 'touch screen' in value_lower or 'touchscreen' in value_lower:
            return 'Yes'
    return 'No'

ebay_df['Touch Screen'] = ebay_df['Screen Size'].apply(check_touch_screen)


Created a new column with cleaned screen sizes:

In [15]:
#this function removes any text and extracts the first number from the rows in the 'Screen Size' column
def extract_first_number(text):
    text = str(text).replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text) #Goes through the text to find a number
    if numbers:
        first_number = float(numbers[0])
        return f"{first_number:.1f} in" #Return the number and in a uniform manner
    return "Unknown"

ebay_df['Cleaned Screen Size'] = ebay_df['Screen Size'].apply(extract_first_number)
ebay_df['Cleaned Screen Size'].replace(to_replace=['1000.0'], 
       value='Unknown', inplace=True)

Extracted only the actual rating from the 'Rating' column  
Left the missing data as NaN so that the column can be a float so that it can be used to do caluclations if needed  
Despite having the majority of its data missing, I believe rating is an important factor for buying/selling laptops so any little bit helps

In [16]:
ebay_df['Rating'] = ebay_df['Rating'].str.split(' ').str[0]
ebay_df['Rating'] = (ebay_df['Rating']).astype(float)


After checking the 'Ratings Count' column, it will be left the way it is so that it can also be used for calculations if needed  
This is also left to help support the rating in the 'Ratings' column

In [17]:
ebay_df['Ratings Count'].unique()

array([      nan, 1.930e+02, 2.300e+01, 2.000e+01, 5.000e+00, 1.000e+00,
       1.000e+01, 8.000e+00, 2.000e+00, 6.000e+00, 3.000e+00, 1.200e+01,
       9.000e+00, 4.100e+01, 5.000e+01, 1.600e+01, 1.533e+03, 4.000e+00,
       9.300e+01, 1.100e+01, 3.100e+01, 1.170e+02, 7.500e+01, 5.300e+01,
       3.600e+01, 1.700e+01, 6.700e+01, 1.500e+01, 4.900e+01, 1.020e+02,
       7.100e+01, 2.600e+01, 2.800e+01, 3.200e+01, 1.030e+02, 4.600e+01,
       2.900e+01, 2.400e+01, 7.000e+00, 2.100e+01, 4.000e+01, 1.190e+02])

Cleaned the 'Ram Size' column by first looking at the uniques and then replacing data where needed:

In [18]:
ebay_df['Ram Size'].unique()

array(['8 GB', nan, '16 GB', '4 GB', '2GB', '32 GB', '4 MB', 'Up to 32GB',
       '12 GB', '256 GB', '1 GB', '20 GB', '3 GB', '16GB', '4GB', '8GB',
       'Up to 64GB', '24 GB', '16GB DDR4 RAM', '16gb', '2 GB', '6 GB',
       '64 GB', '8 GB DDR4', '8GB / 16 GB OR 32GB SELECT FROM ABOVE',
       '8 GB RAM', '8GB,  16GB,  32GB,  64GB', '8192MB', 'Unknown',
       '512 GB', '20gb', 'Up to 36 GB', 'Not Included', '16 GB RAM',
       'up to 32GBGB DDR4', '4gb', '64 MB', '12GB', '8GB DDR4',
       '8 GB DDR4 Installed---Can upgrade o 32GB DDR4', '8', '32GB',
       '20GB', '8 GB 16 GB 32 GB', '1GB', '128 GB', '12gb', '1Gb',
       '40 GB', 'Up to 40GB', 'Up to 32 GB', '4 GB DDR4', '4GB - 16 GB',
       '24GB', '512 MB', '16 GB DDR4', '4 GB- 8 GB'], dtype=object)

In [19]:
#Assumed MB sized were meant to be GB and that ranges were unknown
ebay_df['Ram Size'].replace(to_replace=['Up to 64GB', '64 MB'], 
       value='64 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['Up to 32GB', 'Up to 32 GB', 'up to 32GBGB DDR4', '32GB'], 
       value='32 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['8GB', '8GB DDR4', '8 GB DDR4 Installed---Can upgrade o 32GB DDR4', '8gig', '8 GB RAM', '8 GB DDR4', '8', '8gb'], 
       value='8 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['16 GB DDR4', '16GB', '16 GB RAM', '16GB DDR4 RAM', '16gb'], 
       value='16 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['12gb', '12GB'], 
       value='12 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['Up to 36 GB'], 
       value='36 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['Up to 40GB'], 
       value='40 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['24GB'], 
       value='24 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['20gb', '20GB'], 
       value='20 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['1Gb'], 
       value='1 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['512 MB'], 
       value='512 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['4 MB', '4gb', '4GB', '4 GB DDR4'], 
       value='4 GB', inplace=True)
ebay_df['Ram Size'].replace(to_replace=['8GB,  16GB,  32GB,  64GB', '8192MB', '8 GB 16 GB 32 GB', '4GB - 16 GB', '4 GB- 8 GB','Not Included', '8GB / 16 GB OR 32GB SELECT FROM ABOVE'], 
       value='Unknown', inplace=True)
#fill in remaining missing data with unknown
ebay_df['Ram Size'] = ebay_df['Ram Size'].astype(str).replace('nan', 'Unknown')

Cleaned the 'OS' column by first looking at the uniques and then replacing data where needed:

In [20]:
ebay_df['OS'].unique()

array(['Windows 11 Home', 'Windows 11 Pro', 'Windows 10 Pro', 'Chrome OS',
       'Windows 10', 'win 11 pro', 'Not Included', 'FreeDOS',
       'Windows 10 Home', nan, 'Windows Home 11', 'Windows Pro', 'No Os',
       'Linux', 'Windows 10 Pros', 'Ubuntu', 'Windows 11 Home S',
       '\u200eWindows 11 Home', 'Windows 11 pros', 'Windows 10 pros',
       'Windows', 'Windows 7', 'i5-8265u', 'Windows 11', 'Android',
       'Windows 10 Pro 64Bit', 'Windows 7 Professional', 'macOS Ventura',
       'Windows XP', 'No OS', 'windows 11', 'Win 10',
       'Microsoft Windows 7 Professionnal', 'Windows 10/11 Pro',
       'Windows 10 professionals', 'Professional, Windows 10 Pro',
       'Windows 8.1', 'macOS Big Sur', 'Windows XP 64 Bit',
       'Windows11 Pro', 'Win 10 Pro 64', 'Chrome', 'Not included',
       'Windows 8.1, Windows 7', 'Google Chrome OS',
       'Window 10 Pro Not Activated', 'Windows 10 Professional',
       'Windows 10 Home 64', 'Windows 10 S', 'macOS', 'Kali',
       'Windows XP

In [21]:
ebay_df['OS'].replace(to_replace=['\u200eWindows 11 Home', 'win 11 pro', 'Window 11 Pro', 'Microsoft Windows 11', 'Windows 11 Home S', 'Windows 11 Home', 'Windows 11 Pro', 
                                 'Windows 11 S mode', 'Windows11 Pro', 'Windows 11 pros', 'Windows Home 11', 'Windows 11 Homex64','Windows 11 S Mode', 'windows 11'], 
       value='Windows 11', inplace=True)
ebay_df['OS'].replace(to_replace=['Win 10 Pro 64', 'Windows 10 Home 64',
       'Microsoft Windows 10 Professional', 'Windows 10 pers', 'Professional, Windows 10 Pro', 'Window 10 Pro Not Activated','Professional, Windows 10', 
       'Windows10','Windows 10 S', 'Windows 10 Pro 64Bit', 'Win 10 Pro','Windows 10 Pro Academic', 'Windows 10 professionals', 'Windows 10 Home in S Mode', 
       'Windows 10 Pros', 'Windows 10 Professional', 'Windows 10 pros', 'Win 10', 'Windows 10 Pro', 'Home, Windows 10', 'Windows 10 Home'], 
       value='Windows 10', inplace=True)
ebay_df['OS'].replace(to_replace=['Windows 7 Professional', 'Microsoft Windows 7 Professionnal', 'Windows 7 Pro 64BIT'], 
       value='Windows 7', inplace=True)
ebay_df['OS'].replace(to_replace=['Windows 7 Professional, Windows 10', 'Windows 10/11 Pro', 'Windows', 'Windows Pro', 'No Os','Windows 8.1, Windows 7', 'No OS', 'i5-8265u', 'Android', 'Not included','Windows XP / Windows 7 Pro / Windows 10 Pro'], 
       value='Unknown', inplace=True)
ebay_df['OS'].replace(to_replace=['Mac Os','Mac OS', 'macOS Ventura', 'MacOS', 'macOS Big Sur'], 
       value='macOS', inplace=True)
ebay_df['OS'].replace(to_replace=['Ubuntu/Linux+2nd OS', 'Ubuntu'], 
       value='Ubuntu', inplace=True)
ebay_df['OS'].replace(to_replace=['linux Mint'], 
       value='Linux Mint', inplace=True)
ebay_df['OS'].replace(to_replace=['Windows XP 64 Bit', 'Windows XP Professional Sp3'], 
       value='Windows XP', inplace=True)
ebay_df['OS'].replace(to_replace=['Kali'], 
       value='Kali Linux', inplace=True)
ebay_df['OS'].replace(to_replace=['Chrome OS', 'Chrome', 'Google Chrome OS'], 
       value='Google Chrome', inplace=True)
ebay_df = ebay_df[ebay_df['OS'] != 'Not Included']
#fill in remaining missing data with unknown
ebay_df['OS'] = ebay_df['OS'].astype(str).replace('nan', 'Unknown')



Cleaned the 'Condition' column by first looking at the uniques and then replacing data where needed:

In [22]:
ebay_df['Condition'].unique()

array(["New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is ...  Read moreabout the conditionNew: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item is handmade or was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details. See all condition definitionsopens in a new window or tab ",
       "Very Good - RefurbishedThe item shows minimal wear and is backed by a one year warranty. It is fully functional and has been professionally refurbished,  inspected and cleaned to very good condition by qualified sellers. The item includes original or new accessories and will come in new generic packaging. See the seller's listing for full details. See all condition definitionsopens in a new window or tab",
       'UsedAn item that has been u

In [23]:
ebay_df['Condition'].replace(to_replace=['UsedAn item that has been used previously. The item may have some signs of cosmetic wear, but is fully operational and functions as intended. This item may be a floor model or store return that has been used. See the seller’s listing for full details and description of any imperfections. See all condition definitionsopens in a new window or tab'], 
       value='Used: An item that has been used previously. The item may have some signs of cosmetic wear, but is fully operational and functions as intended. This item may be a floor model or store return that has been used. See the seller’s listing for full details and description of any imperfections. See all condition definitionsopens in a new window or tab', inplace=True)
ebay_df['Condition'].replace(to_replace=["Open boxAn item in excellent, new condition with no wear. The item may be missing the original packaging or protective wrapping, or may be in the original packaging but not sealed. The item includes original accessories. The item may be a factory second. See the seller's listing for full details and description. See all condition definitionsopens in a new window or tab"], 
       value="Open box: An item in excellent, new condition with no wear. The item may be missing the original ...  Read moreabout the conditionOpen box: An item in excellent, new condition with no wear. The item may be missing the original packaging or protective wrapping, or may be in the original packaging but not sealed. The item includes original accessories. The item may be a factory second. See the seller's listing for full details and description. See all condition definitionsopens in a new window or tab ", inplace=True)
ebay_df['Condition'].replace(to_replace=['For parts or not workingAn item that does not function as intended and is not fully operational. This includes items that are defective in ways that render them difficult to use, items that require service or repair, or items missing essential components. See the seller’s listing for full details. See all condition definitionsopens in a new window or tab'], 
       value='For parts or not working: An item that does not function as intended and is not fully operational. ...  Read moreabout the conditionFor parts or not working: An item that does not function as intended and is not fully operational. This includes items that are defective in ways that render them difficult to use, items that require service or repair, or items missing essential components. See the seller’s listing for full details. See all condition definitionsopens in a new window or tab ', inplace=True)
#fill in remaining missing data with unknown
ebay_df['Condition'] = ebay_df['Condition'].astype(str).replace('nan', 'Unknown')

Below is a function where I end up making a new column with shortened condition values and later this will replace the original 'Condition' column. I did this because I do not belive there is a need for the rest of the text description. That can be looked up if necessary.  

In [24]:
# Convert 'Condition' column to string type
ebay_df['Condition'] = ebay_df['Condition'].astype(str)

def extract_before_special_char(text):
    # Split the string at the first occurrence of ":" or " - "
    if " - " in text:
        return text.split(" - ")[0]
    elif ":" in text:
        return text.split(":")[0]
    else:
        return text #Return the first part of the condition description, usually one or a few words to sum up the overall condition

ebay_df['Condition Shortened'] = ebay_df['Condition'].apply(extract_before_special_char)

Cleaned the 'Type' column by first looking at the uniques and then replacing/grouping data where needed:

In [25]:
ebay_df['Type'].unique()

array(['Notebook/Laptop', 'Laptop', 'Netbook', nan,
       'N/A, Notebook/Laptop', 'Convertible 2-in-1 Laptop/Tablet',
       'Notebook', 'Chromebook', 'Ultra-mobile PC (UMPC)', 'NB',
       'Gaming Laptop Computers', 'Windows Laptop', 'Tablet', 'Wireless',
       'Toughbook', '2-in-1 tablet laptop', 'Thinkpad', 'Ultrabook',
       'NOTEBOOK', 'Panel PC', 'Convertible', 'Touchscreen laptop',
       'LCD Display', 'Convertible 2-In-1 Laptop/Tablet',
       'Laptop/Notebook', 'eBook Reader', 'Tablet Computer',
       'Rugged Tough Tablet', 'Getac', 'Laptop/Tablet',
       'Subnotebook/Ultraportable', 'ChromeBook',
       'Ultra Rugged Toughbook', '2-in-1 Convertible', 'Computers',
       '2 in 1 Laptop/Tablet', 'Laptops', 'Workstation/Laptop',
       'Rugged Tablet', 'Tablet/Laptop Convertible', 'Desktop', 'laptop',
       'Macbook Pro', 'Performance Base Keyboard', 'CARDBUS ADAPTER',
       '2 IN 1 CHROMEBOOK', 'Mini Tower PC', 'Tablet NoteBook',
       'Gaming Laptop', 'Keyboard', 'tou

In [26]:
#Since this data is all about laptops, it should be assumed that everything is a laptop, so if there is a two in one 
# then it will be labeled as the option other than laptop (as seen with the tablet labels). Anything remaining will then be labeled as a laptop
ebay_df['Type'].replace(to_replace=['NB', 'N/A, Notebook/Laptop', 'NOTEBOOK', 'Laptop/Notebook', 'Netbook','Subnotebook/Ultraportable', 'Notebook/Laptop'], 
       value='Notebook', inplace=True)
ebay_df['Type'].replace(to_replace=['Laptop/Tablet', 'Tablet', '2-in-1 Convertible'
       '2 in 1 Laptop/Tablet', 'Tablet/Laptop Convertible',
       'Convertible 2-in-1 Laptop/Tablet', 'Tablet Computer','Convertible', 'Convertible 2-In-1 Laptop/Tablet','Touchscreen laptop', 'Notebook/Laptop/Tablet', 'Rugged Tough Tablet', 'Rugged Tablet','Tablet NoteBook'], 
       value='Tablet', inplace=True)
ebay_df['Type'].replace(to_replace=['Laptop/Portable Workstation', '2 IN 1 CHROMEBOOK', 'Computers', 'Laptops', 'Workstation/Laptop', 
       'laptop', 'Macbook Pro','2-in-1 tablet laptop','2-in-1 Convertible', 'Windows Laptop'], 
       value='Laptop', inplace=True)
ebay_df['Type'].replace(to_replace=['Gaming Laptop Computers', 'Gaming Laptop'], 
       value='Gaming', inplace=True)
ebay_df['Type'].replace(to_replace=['Ultra Rugged Toughbook', 'toughpad'], 
       value='Toughbook', inplace=True)
ebay_df['Type'].replace(to_replace=['ChromeBook'], 
       value='Chromebook', inplace=True)
#made a list of keywords to search for and then remove those rows if it seems like the keyword is not a type of laptop
keywords_list = ['Rugged Tablet PC', 'CARDBUS ADAPTER', 'Wireless', 'Desktop','Mini Tower PC', 
       'LCD Touchscreen Assembly','Panel PC', 'LCD Display', 'Performance Base Keyboard', 'Keyboard', 'AWESOME']
for keywords in keywords_list:
    ebay_df = ebay_df[ebay_df['Type'] != keywords]

#fill in remaining missing data with unknown
ebay_df['Type'] = ebay_df['Type'].astype(str).replace('nan', 'Unknown')

Cleaned the 'Color' column by first looking at the uniques and then replacing data where needed as well as grouping the colors together to only have the basic versions of colors:

In [27]:
ebay_df['Color'].unique()

array(['Gray', 'Black', nan, 'Gray, Granite Gray', 'Silver',
       'Natural Silver', 'Dark Brown', 'Carbon Black', 'Grey/Black',
       'Multi', 'Arctic Grey', 'Blue', 'Multicolor', 'Black/Gray',
       'black', 'Mica Silver', 'Steam Blue', 'Purple',
       'Dark Metallic Moon', 'Black/ Blue / Sandtone / Platinum',
       'Dark Grey', 'Silver, Metallic Silver', 'Pink', 'Black / Grey',
       'Grey', 'Beige', 'Red', 'White', 'Gray and Black', 'Silver/Black',
       'Standard', 'Abyss Blue', 'Silver, Platinum Silver', 'Green',
       'Brown', 'Cloud Grey', 'Warm Gold', 'Black/Blue', 'Storm Gray',
       'Mineral Grey', 'Yellow', 'Orange', 'see pictures',
       'Silver + Black', 'Pure Silver', 'Platinum', 'Black / Silver',
       'Immaculate', 'Charcoal Gray', 'Sandstone', 'Fog Blue Aluminum',
       'Matte Black', 'Bronze', 'Silver / Black', 'Gray, Platinum Silver',
       'Silver & Black', 'Gray / Silver', 'STEEL GRAY', 'Gray, Blue',
       'Gold', 'Jet Black', 'Teal', 'Platinum Silve

In [28]:
#grouped some colors together based on similarity or what the first main color mentioned was
ebay_df['Color'].replace(to_replace=['Gray','Gray, Granite Gray','Grey/Black','Cloud Grey','Storm Gray', 'Mineral Grey','Dark Metallic Moon', 'Dark Grey','Gun Metal', 'Gray, Blue','Gray, Platinum Silver','Gray / Silver','Space Gray','Charcoal Gray','Gray and Black', 'STEEL GRAY', 'Arctic Grey'], 
       value='Grey', inplace=True)
ebay_df['Color'].replace(to_replace=['Multi', 'Multicolor', 'Standard', 'Immaculate', 'see pictures', 'Black/ Blue / Sandtone / Platinum'], 
       value='Unspecified', inplace=True)
ebay_df['Color'].replace(to_replace=['Silver', 'Silver + Black','Silver, Platinum Silver','Platinum','Mica Silver','Silver, Metallic Silver','Silver, Black','Silver & Black','Platinum Silver','Pure Silver', 'Mineral Silver', 'Silver/Black', 'Silver and Black', 'Natural Silver', 'Silver / Black'], 
       value='Silver', inplace=True)
ebay_df['Color'].replace(to_replace=['Abyss Blue', 'Steam Blue','Quiet Blue', 'Teal', 'Fog Blue Aluminum'], 
       value='Blue', inplace=True)
ebay_df['Color'].replace(to_replace=['Carbon Black','Black/Gray','Black/Blue','Black / Silver','Matte Black','Black / Gray','Tech Black', 'black', 'Jet Black','Black/Red','Black / Grey'], 
       value='Black', inplace=True)
ebay_df['Color'].replace(to_replace=['Warm Gold'], 
       value='Gold', inplace=True)
ebay_df['Color'].replace(to_replace=['Dark Brown'], 
       value='Brown', inplace=True)
ebay_df['Color'].replace(to_replace=['Sandstone'], 
       value='Yellow', inplace=True)
#fill in remaining missing data with unknown
ebay_df['Color'] = ebay_df['Color'].astype(str).replace('nan', 'Unspecified')


Cleaned the 'Brand' column by first looking at the uniques and then replacing data where needed:

In [29]:
x = ebay_df['Brand'].unique()
x.sort()
x

array(['?', 'ACER', 'AIRBAR', 'AMIAMO', 'ASUS', 'AUUSDA', 'Acer',
       'Alienware', 'Apple', 'CHUWI', 'CLEVO', 'Chuwi', 'Craig', 'DELL',
       'DELL/HP/LENOVO/Fujitsu/Toshiba', 'Dell', 'Dell 16gb', 'Dell Inc.',
       'Dell Inspiron', 'Dell/HP/Lenovo/Stone', 'Different', 'Fujitsu',
       'Fujitsu PC Corp.', 'Fujitsu Siemens', 'Fusion5', 'GIGABYTE',
       'GPD', 'Gateway', 'Getac', 'GoldBook', 'Google', 'HP',
       'HP DELL LENOVO SELECT FROM ABOVE', 'HiDes', 'Huawei', 'IBM',
       'Intel', 'Jumper', 'Kano', 'LENOVO', 'LG', 'Lenovo', 'Lenovo T440',
       'Lenovo T530', 'MSI', 'Metabox', 'Microsoft', 'Olpc', 'Panasonic',
       'ProBook', 'Razer', 'SGIN', 'Samsung', 'Simpletek', 'Sony',
       'Top Performance', 'Toshiba', 'Unbranded', 'VAIO',
       'Vulcan Electronics', 'XPLORE', 'Zebra', 'Zzetze', 'dell',
       'www.TAG1000Diver.com', 'xnote'], dtype=object)

In [30]:
ebay_df['Brand'].replace(to_replace=['Dell Inc.', 'Dell Inspiron','Dell 16gb', 'DELL', 'dell'], 
       value='Dell', inplace=True)
ebay_df['Brand'].replace(to_replace=['Lenovo T440', 'Lenovo T530', 'LENOVO'], 
       value='Lenovo', inplace=True)
ebay_df['Brand'].replace(to_replace=['CHUWI'], 
       value='Chuwi', inplace=True)
ebay_df['Brand'].replace(to_replace=['Fujitsu Siemens', 'Fujitsu PC Corp.'], 
       value='Fujitsu', inplace=True)
ebay_df['Brand'].replace(to_replace=['ACER'], 
       value='Acer', inplace=True)

#made a list of keywords to search for and then remove those rows if it seems like the keyword is not a major brand or not specific
keywords_list = ['Zzetze', 'www.TAG1000Diver.com', 'AIRBAR','Simpletek', 'XPLORE','HiDes','Olpc','GIGABYTE', 'xnote', 'Unbranded', 'Top Performance', 'Zebra','Dell/HP/Lenovo/Stone', 'Different', 'DELL/HP/LENOVO/Fujitsu/Toshiba', '?','HP DELL LENOVO SELECT FROM ABOVE']

for keywords in keywords_list:
    ebay_df = ebay_df[ebay_df['Brand'] != keywords]




Cleaned the 'SSD Capacity' column by first looking at the uniques and then replacing data where needed:

In [31]:
ebay_df['SSD Capacity'].unique()

array(['1 Used,  1 support Max 1TB M.2 Extend', nan, '500 GB', '256 GB',
       '16 GB', '16GB', '64 GB', '128 GB', '32 GB', '512 GB', '1 TB',
       'Up to 2TB', '120 GB', 'See Options', '240GB', '32GB', '480GB',
       '120GB', '240 GB', '256GB', '960GB', '2TB', '4TB', '128GB', '1tb',
       '360GB', 'na', '500GB', '0', '250 GB', '64gb', 'NONE', '2 TB',
       '128GB,  256GB,  500GB,  1TB,  2TB', '4 tb', 'Not Applicable',
       'Up to 1 TB', '1 TB(not in)', '512GB', 'No SSD', '1TB + 1TB',
       '4 GB', 'up to 1TB SSD', '500 GB HDD', '931.51GB', '500gb',
       '256 GBSSD', '16 GB SSD', '800gb', '512', '256gb', '180GB',
       '476GB', 'Up to 2 TB', '120GB - 1TB', '160 GB', '256SSD', 'NO',
       '1024 GB', '512 GB SSD', '115gb', '120', '64GB Solid State Drive',
       '512gb', '120 GB - 500 GB'], dtype=object)

In [32]:
#if the capacity was not specific or obvious then the value was turned into unknown 
ebay_df['SSD Capacity'].replace(to_replace=['512 GB SSD', '512', '512gb', '512GB SSD', '512GB', '512 GB SSD Solid State Drive', '512 GB NVMe M.2 SSD Solid State Drive'], 
       value='512 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['No HDD', 'Not Included', 'NO', 'No', 'Not Applicable', '120GB 240GB 512GB 1TB SELECT FROM ABOVE', '0 GB', 'SEE SSD'
       '3500.16 GB', 'up to 1 TB', 'Not Include', 'NONE', 'None/Unknown','See Options', 'No SSD', '120GB - 1TB',
       '1 TB(not in)','256GB/512GB/1TB SSD (as selected)', 'na','256GB - 500GB - 1TB - 2TB', 'non', 'N/a', 'not included', '256 GB 512 GB 1TB 2TB', 'SSD only', '1',
       '256 GB 512 GB 1 TB 2 TB', '0','80GB-100 GB','SSD','931.51GB','3500.16 GB','SSD Only','120 GB - 500 GB','128GB,  256GB,  500GB,  1TB,  2TB', '64GB', '1TB SSD','250 GB or More','250 GB- 1 TB', '128 GB 256 GB 512 GB 1TB 2TB','SEE SSD','SSD ONLY', '0GB', '250 GB - 1 TB', 'Non', '.', 'Not included', 'N.A.', "Don't Apply", '120 GB or More',
       '320 GB- 500 GB', 'NO HDD'], 
       value='Unknown', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['500GB Solid State Hybrid Drive', '500 GB Large storage memory', '500GB', '500 GB 1 TB 2 TB','500 GB HDD', '500gb'], 
       value='500 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['Up to 2TB', '1TB + 1TB','Up to 2 TB', '2TB m.2 NvMe SSD', '2 TB SSD Solid State Drive', '2TB'], 
       value='2 TB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['16GB', '16 GB SSD'], 
       value='16 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['160GB'], 
       value='160 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['115gb'], 
       value='115 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['180GB'], 
       value='180 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['800gb'], 
       value='800 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['480GB'], 
       value='480 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['32GB'], 
       value='32 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['320GB'], 
       value='320 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['360GB'], 
       value='360 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['960GB'], 
       value='960 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['476GB'], 
       value='476 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['80GB'], 
       value='80 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['400GB'], 
       value='400 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['up to 1TB SSD','1 TB SSD Solid State Drive', '1 Used,  1 support Max 1TB M.2 Extend','1tb nvme', 'Up to 1 TB', '1tb', '1TB'], 
       value='1 TB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['256 GB SSD', '256SSD','256 GBSSD', '256GB', '81GB', '256gb'], 
       value='256 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['128GB', '128 GB (gigabyte)', '128 SSD'], 
       value='128 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['64gb', '64GB eMMC', '64GB SSD', '64GB Solid State Drive'], 
       value='64 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['4 tb', '4TB'], 
       value='4 TB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['120GB', '120'], 
       value='120 GB', inplace=True)
ebay_df['SSD Capacity'].replace(to_replace=['240GB'], 
       value='240 GB', inplace=True)

#fill in remaining missing data with unknown
ebay_df['SSD Capacity'] = ebay_df['SSD Capacity'].astype(str).replace('nan', 'Unknown')


Cleaned the 'Storage Type' column by first looking at the uniques and then replacing data where needed:

In [33]:
ebay_df['Storage Type'].unique()

array(['SSD (Solid State Drive)', 'eMMC', nan, 'HDD + SSD',
       'HDD (Hard Disk Drive)', 'SSD', 'SSD (Condition solid)',
       'UFS (Universal Flash Storage)', 'ssd', 'na', 'Hard Drive', 'HDD',
       'NVME/HDD', 'NVMe', 'M.2', 'M.2 SOLID STATE', 'SSD + eMMC',
       'eMMC Solid State SSD', 'Touchscreen', 'M.2 NVMe', '256GBSSD',
       '256 G', 'eUFS (embedded Universal Flash Storage)', 'other',
       'NVME M.2 SSD (Solid State Drive)',
       'NVMe (Non-Volatile Memory Express)', 'Solid State Drive (SSD)',
       'SSHD', '50GB SSHD (Solid State Hybrid Drive)',
       'SSD (Solid State Drive), HDD + SSD', 'Solid State Drive',
       'HDD / SSD', 'Nvme SSD', 'Flash Memory',
       'SSHD (Solid State Hybrid Drive)', 'SSD + HDD', 'NVMe m.2'],
      dtype=object)

In [34]:
ebay_df['Storage Type'].replace(to_replace=['SSD (Solid State Drive)', '256GBSSD','ssd','SSD (Condition solid)','Solid State Drive','M.2 SOLID STATE','Solid State Drive (SSD)'], 
       value='SSD', inplace=True)
ebay_df['Storage Type'].replace(to_replace=['HDD (Hard Disk Drive)'], 
       value='HDD', inplace=True)
ebay_df['Storage Type'].replace(to_replace=['NVMe m.2', 'M.2','M.2 NVMe','NVME/HDD','Nvme SSD','NVME M.2 SSD (Solid State Drive)',
       'NVMe (Non-Volatile Memory Express)'], 
       value='NVMe', inplace=True)
ebay_df['Storage Type'].replace(to_replace=['eMMC Solid State SSD', 'SSD + eMMC', 'SSD/eMMC'], 
       value='SSD/eMMC', inplace=True)
ebay_df['Storage Type'].replace(to_replace=['UFS (Universal Flash Storage)', 'Flash Memory','eUFS (embedded Universal Flash Storage)'], 
       value='Flash', inplace=True)
ebay_df['Storage Type'].replace(to_replace=['other', 'na'], 
       value='Unknown', inplace=True)
ebay_df['Storage Type'].replace(to_replace=['HDD + SSD','HDD / SSD','SSD + HDD', 'SSHD','SSD (Solid State Drive), HDD + SSD', 'SSHD (Solid State Hybrid Drive)', '50GB SSHD (Solid State Hybrid Drive)'], 
       value='SSD/HDD', inplace=True)
ebay_df = ebay_df[ebay_df['Storage Type'] != 'Touchscreen']
ebay_df = ebay_df[ebay_df['Storage Type'] != '256 G']
ebay_df = ebay_df[ebay_df['Storage Type'] != '.']

#fill in remaining missing data with unknown
ebay_df['Storage Type'] = ebay_df['Storage Type'].astype(str).replace('nan', 'Unknown')


Made a function to only pull out the number for the speed and to get rid of any other text and then add "GHz" at the end to make the values uniform


In [35]:
# Convert 'Processor Speed' column to string type
ebay_df['Processor Speed'] = ebay_df['Processor Speed'].astype(str)
def clean_processor_speed(text):
    text = text.lower() # Make everything lowercase to make comparisons easier
    text = text.replace('up to', '').replace('max', '') #Remove unnecessary text
    if '(' in text: #Split the text and take the first part where number is commonly present
        text = text.split('(')[0]
    if '/' in text:
        text = text.split('/')[1]
    if '-' in text:
        text = text.split('-')[1]
    text = text.strip().replace('ghz', ' GHz') 
    for word in text.split():
        try:
            ghz_value = float(word) #If a number is found then return it with the uniform unit
            return f"{ghz_value:.2f} GHz"
        except ValueError:
            continue
    
    return "Unknown" 

ebay_df['Cleaned Speed'] = ebay_df['Processor Speed'].apply(clean_processor_speed)
ebay_df = ebay_df[ebay_df['Cleaned Speed'] != '800.00 GHz']
ebay_df = ebay_df[ebay_df['Cleaned Speed'] != 'nan GHz']



Made a function to clean and group the types of GPU based on the most common values:

In [36]:
# Convert 'Processor Speed' column to string type
ebay_df['GPU'] = ebay_df['GPU'].astype(str)
def clean_gpu_name(text):
    # Make everything lowercase to make comparisons easier
    text = text.lower()
    
    # Group any similar GPUs
    if 'nvidia' in text:
        if 'geforce' in text:
            return 'NVIDIA GeForce'
        elif 'quadro' in text:
            return 'NVIDIA Quadro'
        elif 'rtx' in text or 'gtx' in text:
            return 'NVIDIA RTX/GTX'
    elif 'amd' in text:
        return 'AMD Radeon'
    elif 'intel' in text:
        if 'iris' in text:
            return 'Intel Iris'
        elif 'uhd' in text:
            return 'Intel UHD'
        elif 'hd graphics' in text:
            return 'Intel HD Graphics'
    elif 'qualcomm' in text:
        return 'Qualcomm Adreno'
    elif 'integrated' in text:
        return 'Integrated Graphics'

    # For any unknown patterns, return 'Other'
    return 'Other'

ebay_df['Cleaned GPU'] = ebay_df['GPU'].apply(clean_gpu_name)

Made functions to help group the types of models and to remove the brand name from these values since that should be in a seperate column:

In [37]:
#dictionary for some of the more popular models and occurances of values
model_dict = {
    'Latitude': ['latitude'],
    'Inspiron': ['inspiron'],
    'Precision': ['precision'],
    'Chromebook': ['chromebook'],
    'EliteBook': ['elitebook', 'elitlebook'],
    'Envy': ['envy'],
    'Probook': ['probook'],
    'Victus': ['victus'], 
    'ThinkPad': ['thinkpad'],
    'IdeaPad': ['ideapad'],
    'CHUWI': ['chuwi'],
    'Swift': ['swift'],
    'Aspire': ['aspire'], 
    'Surface': ['surface'],
 }
#Common brands to take out of the model names
common_brands = ['asus', 'hp', 'dell', 'chuwi', 'lenovo', 'fujitsu', 'gateway', 'samsung', 
                 'microsoft', 'acer', 'panasonic', 'razer', 'msi', 'alienware']

# Function to categorize model names based on the dictionary
def categorize_model(text):
    text = text.lower()  # Make everything lowercase to make comparisons easier
    # Loop through the dictionary
    for clean_brand, variations in model_dict.items():
        # Check if any variation is in the text
        for variation in variations:
            if variation in text:
                return clean_brand  # Return the cleaned brand name
    return remove_brands(text, common_brands)  # Return 'Other' if no match is found


# Function to filter out brands and return the rest of the text
def remove_brands(text, brands):
    for brand in brands:
        if brand in text:
            words = text.lower().split()  #Break up the text
            if brand in words: 
                words.remove(brand)  #Take the brand out of the text
                cleaned_item = ' '.join(words).strip() #Return the rest of the text
                return cleaned_item.title()
    return text.title() #Return the model name if no known brands are in the text
    
ebay_df['Cleaned Model'] = ebay_df['Model'].apply(categorize_model)


In [38]:
ebay_df['Cleaned Model'].unique()

array(['Corebook X', 'Latitude', 'Chromebook', 'Various Models', 'T480',
       'CHUWI', '16 Plus 7630', 'Inspiron', 'Elite X2 G4', 'Surface',
       '100E', '15 Essential', 'Gemibook Plus', 'EliteBook',
       'Yoga 11E 3Rd Gen.', 'Probook', 'Legion 5 Pro 16Iah7', 'ThinkPad',
       'E5580', 'T4M32Ut#Aba', '5470', 'Yoga', 'IdeaPad',
       'Toshiba Qosmio', 'Precision', '3301', 'X15', '15-Fa1093Dx',
       'Cf-33', 'Vivobook', 'C731T-C0X8', '3190 2-In-1', 'Compaq 6910P',
       'Flex 5', 'E5440', 'E7270', 'Swift', '15-Dy2795Wm', 'E736', 'T430',
       'Lifebook S781', 'E734', '13', 'Amilo Xa 2528', 'T470', 'E7450',
       'E570', 'T460', 'A574', 'Victus', 'T470S', 'E5430', 'E6430',
       'V15 G3', '82C5', 'L560', 'E744', 'A315-24Pt-R90Z', 'C731',
       'Xps 15 7590', 'Aspire', 'Lg Gram', 'Revolve 810 G2', 'E590',
       '17B', 'X7-Z8700', 'Google Pixelbook Go', 'Vivobook Flip 14',
       '11 Pro 7130', 'T440S', 'Toughbook Cf-54', '12 Rugged Tablets',
       'X240', 'E6520', 'E546', 

Cleaned the 'Maximum Resolution' column by first looking at the uniques and then replacing data where needed:

In [39]:
ebay_df['Maximum Resolution'].unique()

array(['2160 x 1440', '1920 x 1080', '1366 x 768', nan, '2560 x 1600',
       '3000 x 2000', '2256 x 1504', '1366x768', '2736 x 1824',
       '1920 x 1200', '2560 x 1440', '1280 x 800', 'HD', 'Full HD',
       '1600 x 900', '3840 x 2160', 'FHD (1920 x 1080)', '1920 x 1280',
       '1024 x 768', '2160 X1440', '2256x1504', '1920x1080',
       '2880 x 1920', 'Full HD 1920 x 1080', '2200 x 1650', '2220x1080',
       '1900 x 1200', '1024 x 600', '1366 x 786', 'Full HD (1920 x 1080)',
       '3000 × 2000', '2560 x 1600 ( WQXGA )', '3456 x 2160',
       '1680 x 1050', '800 x 480', '1600x900', '1920 x 1080 (FHD)',
       '3840 x 2400', '1920 x 1080 ( Full HD )', '1920 x 1080 Full HD',
       '1920 x 1080p', '3200 x 1800', '1800 x 1200', '3072 x 1920',
       '1028 x 800',
       '1366 x 768 pixels at 262 K colors HD WLED Anti-Glare', '1600X900',
       '2880 x 1800', '1280 x 720', '1280 x 1024', '1920x1080 FHD',
       '1900x600', '1366 x 912', '2160 x 1350', '1920X1080', '1024 x 728',
       

In [40]:
ebay_df['Maximum Resolution'].replace(to_replace=['Full HD (1920 x 1080)', '1920x1080 Full HD','1920X1080','1920x1080 FHD','1920 x 1080 ( Full HD )','Full HD 1920 x 1080','FHD (1920 x 1080)','1920 x 1080p', '1920x1080', 'Full HD (1920x1080p)'], 
       value='1920 x 1080', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['1366x768', 'HD (1366 x 768)'], 
       value='1366 x 768', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['2256x1504'], 
       value='2256 x 1504', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['2160 X1440'], 
       value='2160 x 1440', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['3000 × 2000'], 
       value='3000 x 2000', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['3200x1800'], 
       value='3200 x 1800', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['2220x1080'], 
       value='2220 x 1080', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['2560 x 1600 ( WQXGA )'], 
       value='2560 x 1600', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['2560x1600'], 
       value='2560 x 1600', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['1900x600'], 
       value='1900 x 600', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['1920 x 1080 (FHD)', '1920 x 1080 Full HD'], 
       value='1920 x 1080', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['1900x1200'], 
       value='1900 x 1200', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['1920x1200'], 
       value='1920 x 1200', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['1600X900', '1600x900'], 
       value='1600 x 900', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['1366 x 768 ( HD )', '1366X768 HD', '1366 x 768 pixels at 262 K colors HD WLED Anti-Glare'], 
       value='1366 x 768', inplace=True)
ebay_df['Maximum Resolution'].replace(to_replace=['HD', 'Full HD', 'ANY'], 
       value='Unknown', inplace=True)
ebay_df = ebay_df[ebay_df['Maximum Resolution'] != 'High Quality']
#fill in remaining missing data with unknown
ebay_df['Maximum Resolution'] = ebay_df['Maximum Resolution'].astype(str).replace('nan', 'Unknown')

The column 'Processor' was left in its orginial state since by the end of the wrangling it had almost all of its data and it had such unique values, in addtion to there being so many different processors, that it seemed better to be able to leave the specific information the way the seller labeled it. 

Dropping and renaming columns to replace some of the original columns with their newer versions:

In [41]:
ebay_df.drop(columns = ['Model', 'GPU', 'Condition', 'Processor Speed', 'Screen Size'], inplace = True)
ebay_df.rename(columns={'Cleaned Screen Size': 'Screen Size', 
 'Cleaned Speed': 'Processor Speed', 'Cleaned GPU': 'GPU', 'Cleaned Model': 'Model', 'Min Price': 'Min Price ($)', 'Max Price': 'Max Price ($)'}, inplace=True)
pd.set_option("display.max_columns", None)
ebay_df

,Item Number,Brand,Rating,Ratings Count,Processor,Color,Ram Size,SSD Capacity,Type,Release Year,Maximum Resolution,OS,Hard Drive Capacity,Storage Type,Min Price ($),Max Price ($),Refurbished,Touch Screen,Screen Size,Condition Shortened,Processor Speed,GPU,Model
0,85264981001,Chuwi,NaN,NaN,Quad Core,Grey,8 GB,1 TB,Notebook,2021,2160 x 1440,Windows 11,512 GB,SSD,303.68,303.68,No,No,14.0 in,New,3.80 GHz,Intel Iris,Corebook X
1,85264981002,Dell,NaN,NaN,Intel Core i7 8th Gen.,Black,Unknown,Unknown,Notebook,Unknown,1920 x 1080,Windows 11,2 TB,SSD,399.99,634.99,Yes,No,14.0 in,Very Good,4.20 GHz,Intel UHD,Latitude
2,85264981003,Dell,NaN,NaN,Intel Core i5-6300U,Black,16 GB,500 GB,Notebook,2019,1920 x 1080,Windows 10,500 GB,SSD,175.00,175.00,No,No,14.0 in,Used,2.40 GHz,Intel HD Graphics,Latitude
3,85264981004,HP,NaN,NaN,Intel Celeron N,Black,4 GB,Unknown,Notebook,Unknown,1366 x 768,Google Chrome,16 GB,eMMC,84.99,84.99,No,No,11.6 in,Good,2.40 GHz,Intel HD Graphics,Chromebook
4,85264981005,Dell,NaN,NaN,Intel Core i5 6th Gen.,Unspecified,8 GB,256 GB,Notebook,2015,1366 x 768,Windows 10,None/Unknown,SSD,101.22,101.22,No,No,12.5 in,Good,1.40 GHz,Integrated Graphics,Various Models
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6641,85264987586,Dell,NaN,NaN,Intel Core i5 4th generation,Black,Unknown,120 GB,Notebook,Unknown,Unknown,Windows 10,None/Unknown,Unknown,323.66,323.66,No,No,14.0 in,Used,2.60 GHz,Intel HD Graphics,E5440
6650,85264987595,Dell,NaN,NaN,Intel Core i5 3rd generation,Unspecified,Unknown,120 GB,Notebook,Unknown,Unknown,Windows 11,None/Unknown,SSD,270.15,270.15,No,No,14.0 in,Seller refurbished,3.10 GHz,Intel HD Graphics,E5430
6651,85264987596,Dell,NaN,NaN,Intel Core i5 4th generation,Black,Unknown,960 GB,Notebook,Unknown,Unknown,Windows 10,None/Unknown,Unknown,429.10,429.10,No,No,14.0 in,Used,2.60 GHz,Intel HD Graphics,E5440
6657,85264987602,Dell,NaN,NaN,Intel Core i5 6th generation,Black,Unknown,120 GB,Notebook,Unknown,Unknown,Windows 11,None/Unknown,Unknown,424.83,424.83,No,No,12.0 in,Used,2.80 GHz,Intel HD Graphics,E7270


In [42]:
ebay_df.dtypes

Item Number              int64
Brand                   object
Rating                 float64
Ratings Count          float64
Processor               object
Color                   object
Ram Size                object
SSD Capacity            object
Type                    object
Release Year            object
Maximum Resolution      object
OS                      object
Hard Drive Capacity     object
Storage Type            object
Min Price ($)          float64
Max Price ($)          float64
Refurbished             object
Touch Screen            object
Screen Size             object
Condition Shortened     object
Processor Speed         object
GPU                     object
Model                   object
dtype: object

Remove duplicates:

In [43]:
ebay_clean = ebay_df.drop_duplicates()
len(ebay_clean)

2450

In [44]:
ebay_clean.isna().sum() / len(ebay_clean) * 100

Item Number             0.000000
Brand                   0.000000
Rating                 89.673469
Ratings Count          89.673469
Processor               0.816327
Color                   0.000000
Ram Size                0.000000
SSD Capacity            0.000000
Type                    0.000000
Release Year            0.000000
Maximum Resolution      0.000000
OS                      0.000000
Hard Drive Capacity     0.000000
Storage Type            0.000000
Min Price ($)           0.000000
Max Price ($)           0.000000
Refurbished             0.000000
Touch Screen            0.000000
Screen Size             0.000000
Condition Shortened     0.000000
Processor Speed         0.000000
GPU                     0.000000
Model                   0.000000
dtype: float64

Grouping and aggregating the data:  
This can show what the average rating is per brand, the statistics for the prices per brand, and the amount of devices lsited per brand, all of which could be useful information for an analysis
To be more specific, the models could be grouped together

In [48]:
#grouping the brands 
brand_groups = ebay_clean.groupby('Brand')
brand_groups.sum()
brand_groups.agg(['count', 'sum', 'mean', 'max', 'min'])

Item Number                                                          \
                count             sum          mean          max          min   
Brand                                                                           
ASUS               53   4519044193621  8.526498e+10  85264987455  85264981083   
Acer               86   7332788646577  8.526498e+10  85264987123  85264981006   
Alienware           1     85264984453  8.526498e+10  85264984453  85264984453   
Apple              11    937914831414  8.526498e+10  85264987518  85264983121   
CLEVO               1     85264985000  8.526498e+10  85264985000  85264985000   
Chuwi              17   1449504710975  8.526498e+10  85264985056  85264981001   
Craig               1     85264983680  8.526498e+10  85264983680  85264983680   
Dell             1004  85606044401996  8.526498e+10  85264987603  85264981002   
Fujitsu           105   8952823348565  8.526498e+10  85264987576  85264981126   
Fusion5             2    170529970728  8.526499e+10  85264986309  85264984419   
GPD                 1     85264983669  8.526498e+10  85264983669  85264983669   
Gateway            14   1193709779990  8.526498e+10  85264986206  85264983405   
Getac              16   1364239755476  8.526498e+10  85264987124  85264981602   
Google              3    255794948607  8.526498e+10  85264983625  85264981608   
HP                356  30354334436123  8.526498e+10  85264987185  85264981004   
Huawei              2    170529968133  8.526498e+10  85264984433  85264983700   
IBM                 1     85264984461  8.526498e+10  85264984461  85264984461   
Intel              11    937914830753  8.526498e+10  85264987169  85264982127   
Jumper              1     85264984128  8.526498e+10  85264984128  85264984128   
Kano                3    255794953665  8.526498e+10  85264985660  85264983778   
LG                 11    937914827684  8.526498e+10  85264986609  85264981589   
Lenovo            549  46810476406403  8.526498e+10  85264987399  85264981011   
MSI                 7    596854897393  8.526499e+10  85264987134  85264983384   
Metabox             1     85264983737  8.526498e+10  85264983737  85264983737   
Microsoft          80   6821198748170  8.526498e+10  85264987106  85264981026   
Panasonic          31   2643214511081  8.526498e+10  85264986506  85264981077   
ProBook             2    170529970168  8.526499e+10  85264985490  85264984678   
Razer               3    255794953724  8.526498e+10  85264984860  85264984369   
SGIN               13   1108444799977  8.526498e+10  85264987110  85264981072   
Samsung            43   3666394337934  8.526498e+10  85264987148  85264982132   
Toshiba            20   1705299694086  8.526498e+10  85264986903  85264981062   
VAIO                1     85264982138  8.526498e+10  85264982138  85264982138   

          Rating                            Ratings Count                      \
           count    sum      mean  max  min         count     sum        mean   
Brand                                                                           
ASUS          10   46.0  4.600000  5.0  4.5            10   359.0   35.900000   
Acer          29  122.0  4.206897  5.0  3.0            29  1474.0   50.827586   
Alienware      0    0.0       NaN  NaN  NaN             0     0.0         NaN   
Apple          0    0.0       NaN  NaN  NaN             0     0.0         NaN   
CLEVO          0    0.0       NaN  NaN  NaN             0     0.0         NaN   
Chuwi          0    0.0       NaN  NaN  NaN             0     0.0         NaN   
Craig          0    0.0       NaN  NaN  NaN             0     0.0         NaN   
Dell          46  206.5  4.489130  5.0  1.0            46   734.0   15.956522   
Fujitsu        2   10.0  5.000000  5.0  5.0             2    10.0    5.000000   
Fusion5        0    0.0       NaN  NaN  NaN             0     0.0         NaN   
GPD            0    0.0       NaN  NaN  NaN             0     0.0         NaN   
Gateway        6   23.5  3.916667  5.0  

Save as a new file if needed:

In [45]:
#rebay_clean.to_csv('Cleaned_Ebay_Data.csv')